In [1]:
import sympy
mass,hbar,energy=sympy.symbols('m,hbar,E_\\text{eigen}',positive=True)
position=sympy.symbols('x',real=True)
waveFunction,potential,action,density=map(lambda function:function(position),sympy.symbols('\Psi,V,S,varrho',cls=sympy.Function))
schrodinger=-hbar**2/2/mass*waveFunction.diff(position,2)+(potential-energy)*waveFunction
sympy.Eq(schrodinger,0)

        2                                                
    2  d                                                 
  h̅ ⋅───(\Psi(x))                                       
        2                                                
      dx                                                 
- ──────────────── + (-E_\text{eigen} + V(x))⋅\Psi(x) = 0
        2⋅m                                              

In [2]:
sqrtDensity=sympy.Function('sqrtDensity')(position)
jacobi=sympy.use(sympy.expand(schrodinger.subs(waveFunction,sqrtDensity*sympy.exp(sympy.I*action/hbar)).doit()/sympy.exp(sympy.I*action/hbar)),
                 lambda node:0 if node.has(sympy.I) else node/sqrtDensity,level=1).subs(sqrtDensity,sympy.sqrt(density))
jacobi

                        2                                    
                    2  d ⎛  ___________⎞                    2
                  h̅ ⋅───⎝╲╱ varrho(x) ⎠          ⎛d       ⎞ 
                        2                         ⎜──(S(x))⎟ 
                      dx                          ⎝dx      ⎠ 
-E_\text{eigen} - ────────────────────── + V(x) + ───────────
                          ___________                 2⋅m    
                    2⋅m⋅╲╱ varrho(x)                         

In [3]:
flux=sympy.symbols('j',real=True)
velocity=sympy.Function('v')(position)
jacobi=jacobi.subs(density,flux/velocity).subs(action.diff(position),velocity*mass).doit()
jacobi

                      ⎛                            2⎞                 
                      ⎜                  ⎛d       ⎞ ⎟                 
                      ⎜      2         3⋅⎜──(v(x))⎟ ⎟                 
                    2 ⎜     d            ⎝dx      ⎠ ⎟                 
                  h̅ ⋅⎜- 2⋅───(v(x)) + ─────────────⎟                 
                      ⎜      2              v(x)    ⎟      2          
                      ⎝    dx                       ⎠   m⋅v (x)       
-E_\text{eigen} - ─────────────────────────────────── + ─────── + V(x)
                                8⋅m⋅v(x)                   2          

In [4]:
time=sympy.symbols('t',real=True)
position=sympy.Function('x')(time)
position.func.is_real=True
def eulerLagrange(expr):
    def eulerLagrange(expr):
        derivative=position.diff(time)
        if isinstance(expr,sympy.Derivative):
            for order in range(1,len(expr.variables)+1):derivative=derivative.diff(time)/position.diff(time)
        return derivative
    import sys
    if isinstance(expr,sympy.Derivative) or expr==velocity:return eulerLagrange(expr)
    else:return expr.func(*map(sys.modules[__name__].eulerLagrange,expr.args)) if expr.args else expr
jacobi=eulerLagrange(jacobi).expand().subs(potential,potential.func(position)).collect(hbar**2/mass)
potential=potential.func(position)
jacobi

                      ⎛                             2⎞                        
                      ⎜    3             ⎛  2      ⎞ ⎟                        
                      ⎜   d              ⎜ d       ⎟ ⎟                        
                      ⎜  ───(x(t))     5⋅⎜───(x(t))⎟ ⎟                        
                      ⎜    3             ⎜  2      ⎟ ⎟                        
                    2 ⎜  dt              ⎝dt       ⎠ ⎟                        
                  h̅ ⋅⎜───────────── - ──────────────⎟                        
                      ⎜            3               4 ⎟               2        
                      ⎜  ⎛d       ⎞      ⎛d       ⎞  ⎟     ⎛d       ⎞         
                      ⎜4⋅⎜──(x(t))⎟    8⋅⎜──(x(t))⎟  ⎟   m⋅⎜──(x(t))⎟         
                      ⎝  ⎝dt      ⎠      ⎝dt      ⎠  ⎠     ⎝dt      ⎠         
-E_\text{eigen} + ──────────────────────────────────── + ───────────── + V(x(t
                                   m                

In [5]:
quantum=sympy.use(jacobi,lambda node:node if node.has(position.diff(time,2)) or node.has(position.diff(time,3)) else 0,level=1)
quantum

    ⎛                             2⎞
    ⎜    3             ⎛  2      ⎞ ⎟
    ⎜   d              ⎜ d       ⎟ ⎟
    ⎜  ───(x(t))     5⋅⎜───(x(t))⎟ ⎟
    ⎜    3             ⎜  2      ⎟ ⎟
  2 ⎜  dt              ⎝dt       ⎠ ⎟
h̅ ⋅⎜───────────── - ──────────────⎟
    ⎜            3               4 ⎟
    ⎜  ⎛d       ⎞      ⎛d       ⎞  ⎟
    ⎜4⋅⎜──(x(t))⎟    8⋅⎜──(x(t))⎟  ⎟
    ⎝  ⎝dt      ⎠      ⎝dt      ⎠  ⎠
────────────────────────────────────
                 m                  

In [6]:
lagrange=mass/2*position.diff(time)**2-potential-quantum
lagrange

      ⎛                             2⎞                          
      ⎜    3             ⎛  2      ⎞ ⎟                          
      ⎜   d              ⎜ d       ⎟ ⎟                          
      ⎜  ───(x(t))     5⋅⎜───(x(t))⎟ ⎟                          
      ⎜    3             ⎜  2      ⎟ ⎟                          
    2 ⎜  dt              ⎝dt       ⎠ ⎟                          
  h̅ ⋅⎜───────────── - ──────────────⎟                          
      ⎜            3               4 ⎟               2          
      ⎜  ⎛d       ⎞      ⎛d       ⎞  ⎟     ⎛d       ⎞           
      ⎜4⋅⎜──(x(t))⎟    8⋅⎜──(x(t))⎟  ⎟   m⋅⎜──(x(t))⎟           
      ⎝  ⎝dt      ⎠      ⎝dt      ⎠  ⎠     ⎝dt      ⎠           
- ──────────────────────────────────── + ───────────── - V(x(t))
                   m                           2                

In [7]:
motion=sympy.euler_equations(lagrange,position)[0].expand()
motion

                                                                 3            
         4                  2         3               ⎛  2      ⎞             
     2  d               2  d         d              2 ⎜ d       ⎟             
   h̅ ⋅───(x(t))    2⋅h̅ ⋅───(x(t))⋅───(x(t))   5⋅h̅ ⋅⎜───(x(t))⎟             
         4                  2         3               ⎜  2      ⎟        2    
       dt                 dt        dt                ⎝dt       ⎠       d     
- ─────────────── + ───────────────────────── - ────────────────── - m⋅───(x(t
                4                     5                        6         2    
      ⎛d       ⎞            ⎛d       ⎞               ⎛d       ⎞        dt     
  4⋅m⋅⎜──(x(t))⎟          m⋅⎜──(x(t))⎟           2⋅m⋅⎜──(x(t))⎟               
      ⎝dt      ⎠            ⎝dt      ⎠               ⎝dt      ⎠               

                       
                       
                       
                       
                       
       d  

In [8]:
gauge=hbar**2*position.diff(time,2)/4/mass/position.diff(time)**3
gauge

       2       
   2  d        
 h̅ ⋅───(x(t)) 
       2       
     dt        
───────────────
              3
    ⎛d       ⎞ 
4⋅m⋅⎜──(x(t))⎟ 
    ⎝dt      ⎠ 

In [9]:
lagrange+=gauge.diff(time)
lagrange=lagrange.expand()
lagrange

                 2                          
      ⎛  2      ⎞                           
    2 ⎜ d       ⎟                2          
  h̅ ⋅⎜───(x(t))⎟      ⎛d       ⎞           
      ⎜  2      ⎟    m⋅⎜──(x(t))⎟           
      ⎝dt       ⎠      ⎝dt      ⎠           
- ──────────────── + ───────────── - V(x(t))
                4          2                
      ⎛d       ⎞                            
  8⋅m⋅⎜──(x(t))⎟                            
      ⎝dt      ⎠                            

In [10]:
assert(not sympy.simplify(sympy.euler_equations(lagrange,position)[0].expand()-motion))

In [11]:
momentums=([sum(map(lambda derivative:(-1)**derivative*lagrange.diff(position.diff(time,derivative+1)).diff(time,derivative),range(0,2))).expand().collect(hbar**2/mass)]
           +[lagrange.diff(position.diff(time,2))])
momentums

⎡    ⎛                            2⎞                              ⎤
⎢    ⎜    3            ⎛  2      ⎞ ⎟                              ⎥
⎢    ⎜   d             ⎜ d       ⎟ ⎟                              ⎥
⎢    ⎜  ───(x(t))      ⎜───(x(t))⎟ ⎟                              ⎥
⎢    ⎜    3            ⎜  2      ⎟ ⎟                              ⎥
⎢  2 ⎜  dt             ⎝dt       ⎠ ⎟                              ⎥
⎢h̅ ⋅⎜───────────── - ─────────────⎟                      2       ⎥
⎢    ⎜            4               5⎟                  2  d        ⎥
⎢    ⎜  ⎛d       ⎞      ⎛d       ⎞ ⎟               -h̅ ⋅───(x(t)) ⎥
⎢    ⎜4⋅⎜──(x(t))⎟    2⋅⎜──(x(t))⎟ ⎟                      2       ⎥
⎢    ⎝  ⎝dt      ⎠      ⎝dt      ⎠ ⎠     d              dt        ⎥
⎢─────────────────────────────────── + m⋅──(x(t)), ───────────────⎥
⎢                 m                      dt                      4⎥
⎢                                                      ⎛d       ⎞ ⎥
⎢                                               

In [12]:
Momentums=sympy.symbols('p,p1',real=True)
positionDerivatives=sympy.solve(momentums[1]-Momentums[1],position.diff(time,2))
positionDerivatives.extend(sympy.solve(momentums[0].replace(position.diff(time,2),positionDerivatives[0])-Momentums[0],position.diff(time,3)))
positionDerivatives

⎡                  4       ⎛                                           3⎞     
⎢        ⎛d       ⎞        ⎜  2 ⎛    d           ⎞         2 ⎛d       ⎞ ⎟ ⎛d  
⎢-4⋅m⋅p₁⋅⎜──(x(t))⎟    4⋅m⋅⎜h̅ ⋅⎜- m⋅──(x(t)) + p⎟ + 8⋅m⋅p₁ ⋅⎜──(x(t))⎟ ⎟⋅⎜──(
⎢        ⎝dt      ⎠        ⎝    ⎝    dt          ⎠           ⎝dt      ⎠ ⎠ ⎝dt 
⎢────────────────────, ───────────────────────────────────────────────────────
⎢          2                                          4                       
⎣        h̅                                         h̅                        

      4⎤
     ⎞ ⎥
x(t))⎟ ⎥
     ⎠ ⎥
───────⎥
       ⎥
       ⎦

In [13]:
hamiltonian=sympy.expand(sum(map(lambda element:momentums[element]*position.diff(time,element+1),range(0,len(momentums))))-lagrange)
def replace(expr):
    return (expr.subs(hbar,1).subs(mass,sympy.symbols('this->ReactionCoordinate.Mass'))
            .subs(map(lambda derivative:(position.diff(time,derivative),sympy.symbols('Function.at('+str(derivative)+')')),range(3,0,-1))))
print('''#include"lagrange.h"
std::vector<double>trajectory::Lagrange_::Derivative(std::vector<double>const&Function)const&
{
  std::vector<double>derivative(4);
  std::copy(Function.cbegin()+1,Function.cend(),derivative.begin());
  derivative.at(3)='''+sympy.ccode(replace(sympy.solve(motion,position.diff(time,4))[0])
                                   .subs(potential.diff(position).doit(),sympy.symbols('this->ReactionCoordinate.Derivative(Function.front()\,*(Function.cend()-2))')))+''';
  return derivative;
}

double trajectory::Lagrange_::Energy()const&noexcept
{
  auto const Trajectory(this->Get());
  return '''+sympy.ccode(replace(hamiltonian).subs(potential,0)).replace('Function','Trajectory')+''';
}

double trajectory::Lagrange_::Momentum()const&noexcept
{
  auto const Trajectory(this->Get());
  return '''+sympy.ccode(replace(momentums[0])).replace('Function','Trajectory')+''';
}''',file=open('lagrangeDerivative.cpp','w'))
hamiltonian=hamiltonian.subs(map(lambda derivative:(position.diff(time,derivative),positionDerivatives[derivative-2]),range(3,1,-1))).expand()
hamiltonian

              2                                            4
    ⎛d       ⎞                                 2 ⎛d       ⎞ 
  m⋅⎜──(x(t))⎟                           2⋅m⋅p₁ ⋅⎜──(x(t))⎟ 
    ⎝dt      ⎠      d                            ⎝dt      ⎠ 
- ───────────── + p⋅──(x(t)) + V(x(t)) - ───────────────────
        2           dt                             2        
                                                 h̅         

In [14]:
import IPython.display
IPython.display.Latex(r'''
\begin{equation}
\left\{\begin{array}{l}
\dfrac{dx}{dt}=\dfrac{\partial\mathscr{H}}{\partial p}\\
\dfrac{dp}{dt}=-\dfrac{\partial\mathscr{H}}{\partial x}='''+sympy.latex(-hamiltonian.diff(position).doit()).replace('frac','dfrac')+r'''
\end{array}\right.
\left\{\begin{array}{l}
\dfrac{d^2x}{dt^2}=\dfrac{\partial\mathscr{H}}{\partial p_1}='''+sympy.latex(hamiltonian.diff(Momentums[1])).replace('frac','dfrac')+r'''\\
\dfrac{dp_1}{dt}=-\dfrac{\partial\mathscr{H}}{\partial\tfrac{dx}{dt}}='''+sympy.latex(-hamiltonian.diff(position.diff(time))).replace('frac','dfrac')+r'''
\end{array}\right.
\end{equation}''')

<IPython.core.display.Latex object>

In [15]:
assert(not position.diff(time)-hamiltonian.diff(Momentums[0]))
assert(not sympy.expand(momentums[0].diff(time)+hamiltonian.diff(position).doit()+motion.lhs))
assert(not sympy.Subs(position.diff(time,2)-hamiltonian.diff(Momentums[1]),Momentums[1],momentums[1]).doit())
assert(not (momentums[1].diff(time)+hamiltonian.diff(position.diff(time))).subs(zip(Momentums,momentums)).expand())
def replace(hamiltonian):
    return (hamiltonian.subs(hbar,1).subs(mass,sympy.symbols('this->ReactionCoordinate.Mass')).subs(position.diff(time),sympy.symbols('Function.at(2)'))
            .subs(Momentums[1],sympy.symbols('Function.at(3)')))
print('''#include"hamilton.h"
std::vector<double>trajectory::Hamilton_::Derivative(std::vector<double>const&Function)const&
{
  std::vector<double>derivative(4);
  derivative.at(0)=Function.at(2);
  derivative.at(1)=-this->ReactionCoordinate.Derivative(Function.front(),*(Function.cend()-2));
  derivative.at(2)='''+sympy.ccode(replace(hamiltonian.diff(Momentums[1])))+''';
  derivative.at(3)='''+sympy.ccode(replace(-hamiltonian.diff(position.diff(time))).subs(Momentums[0],sympy.symbols('Function.at(1)')))+''';
  return derivative;
}

double trajectory::Hamilton_::Energy()const&noexcept
{
  auto const Trajectory(this->Get());
  return '''+sympy.ccode(replace(hamiltonian).subs(Momentums[0],sympy.symbols('Function.at(1)')).subs(potential,0)).replace('Function','Trajectory')+''';
}

double trajectory::Hamilton_::Acceleration()const&noexcept
{
  auto const Trajectory(this->Get());
  return '''+sympy.ccode(replace(hamiltonian.diff(Momentums[1]))).replace('Function','Trajectory')+''';
}''',file=open('hamiltonDerivative.cpp','w'))

In [16]:
monochrome=sympy.exp(sympy.I*sympy.sqrt(2*mass*energy)/hbar*position)
def velocity(waveFunction):return sympy.trigsimp(sympy.im(waveFunction.diff(position)/waveFunction)*hbar/mass)
reflection,transmission=sympy.symbols('R,T')
product=transmission*monochrome
product.subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{P}'))

          ______________________________        
   √2⋅ⅈ⋅╲╱ E_\text{eigen}-V_\mathscr{P} ⋅√m⋅x(t)
   ─────────────────────────────────────────────
                         h̅                     
T⋅ℯ                                             

In [17]:
velocity(product).subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{P}'))

     ______________________________
√2⋅╲╱ E_\text{eigen}-V_\mathscr{P} 
───────────────────────────────────
                 √m                

In [18]:
reflection=sympy.Abs(reflection)*sympy.exp(sympy.I*sympy.arg(reflection))
reactant=sympy.powsimp(monochrome+reflection*sympy.conjugate(monochrome))
reactant.subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{R}'))

        ______________________________                     ___________________
 √2⋅ⅈ⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅√m⋅x(t)      √2⋅ⅈ⋅╲╱ E_\text{eigen}-V_\
 ─────────────────────────────────────────────    - ──────────────────────────
                       h̅                                                 h̅  
ℯ                                              + ℯ                            

___________                       
mathscr{R} ⋅√m⋅x(t)               
─────────────────── + ⅈ⋅arg(R)    
                                  
                              ⋅│R│

In [19]:
rectantVelocity=velocity(reactant)
rectantVelocity.subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{R}'))

                      ______________________________ ⎛     2    ⎞             
                 √2⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅⎝- │R│  + 1⎠             
──────────────────────────────────────────────────────────────────────────────
   ⎛     ⎛       ______________________________                 ⎞             
   ⎜     ⎜2⋅√2⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅√m⋅x(t)         ⎟          2  
√m⋅⎜2⋅cos⎜───────────────────────────────────────────── - arg(R)⎟⋅│R│ + │R│  +
   ⎝     ⎝                      h̅                              ⎠             

   
   
───
  ⎞
  ⎟
 1⎟
  ⎠

\begin{equation}
\dfrac{d^2x}{dt^2}=\dfrac{d}{dt}\dfrac{dx}{dt}=\dfrac{dx}{dt}\dfrac{d}{dx}\dfrac{dx}{dt}
\end{equation}

In [20]:
momentum=momentums[0]
for _ in range(0,3):momentum=momentum.subs(position.diff(time),rectantVelocity).doit()
momentum=momentum.trigsimp()
momentum.subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{R}'))

      ______________________________    ⎛   2    ⎞ 
-√2⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅√m⋅⎝│R│  + 1⎠ 
───────────────────────────────────────────────────
                         2                         
                      │R│  - 1                     

In [21]:
probability=sympy.symbols('|T|^2',real=True)
momentum=momentum.subs(sympy.Abs(reflection)**2,1-probability)
sympy.solveset(momentum-Momentums[0],probability,sympy.S.Reals).args[0].inf.subs(energy,sympy.symbols('E_\\text{eigen}-V_\mathscr{R}'))

        ______________________________    
 2⋅√2⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅√m 
──────────────────────────────────────────
     ______________________________       
√2⋅╲╱ E_\text{eigen}-V_\mathscr{R} ⋅√m + p